<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/max_ind_set_openmp_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Maximum Independent Set on GPUs using OMP


# THe main pragma directives: target and loop


In [57]:
%%writefile max_ind_set_omp_gpu.cc
/*
 *
 * Solve the Maximum Independent Set problem on GPUs using OpenMP
 *
 * https://en.wikipedia.org/wiki/Independent_set_(graph_theory)
 * This code works for graphs of up to 32 nodes
 * The input is expected to be as follows:
 * The first line will contain n, the number of elements in the multiset
 * The remaining n lines will contain the adjacency matrix
 */
#include <omp.h>
#include <vector>
#include <iostream>
#include <sstream>
#include <string>

using namespace std;

// This functions evaluates a particular subset of nodes.
// adjMat is the adjacency
// n is the size of the number of nodes in the graph
// value is an integer value. The binary code of value
//

int evaluateSubSet(  int value, int n, vector < vector < int> >& adjMat) {
  vector < int > set(n,0);
  unsigned int mask = 1;
  int bitCount = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      set[i] = 1;
      bitCount++;
    }
    mask = mask * 2;
  }
  for(int i = 0;i < n;i++) {
    for(int j = i+1;j < n;j++) {
      if (set[i] == 1 && set[j] == 1 && adjMat[i][j] == 1) {
        return 0;
      }
    }
  }
  return bitCount;
}

void printResults(unsigned int value,int n)
{


  unsigned int mask = 1;
  int sum = 0;
  cout << "An independent set with the maximum number of nodes is: " << endl;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      cout << i << " ";
    }
    mask = mask * 2;
  }
  cout << endl;
}

// The main function
int main() {

  int n;

// The format of the input is
// an integer value n with the size of the multiset
// n integer values with the multiset

  vector < vector < int> > adjMat;
  string line;
  getline(cin,line);
  stringstream ss(line);
  ss >> n;
  // Read input line by line until EOF
  while (std::getline(cin, line)) {
        stringstream ss(line);
        vector<int> row;
        int num;
        while (ss >> num) {
          //cout << num << " ";
          row.push_back(num);
        }
        //cout << "Size of row: " << row.size() << endl;
        adjMat.push_back(row);
        //cout << endl;
  }
  cout << "The size of the graph is: " << n << endl;
  cout << "The number of rows in the adjacency matrix is: " << adjMat.size() << endl;
  cout << "The number of columns in the adjacency matrix is: " << adjMat[0].size() << endl;

// Print the adjacency matrix

  cout << "The adjacency matrix is : " << endl;
  for(int i = 0;i < n;i++) {
    for(int j = 0;j < n;j++) {
      cout << adjMat[i][j] << " ";
    }
    cout << endl;
  }

// Calculate the set of the power set
  unsigned int powerOf2 = 1;
  for(int i = 0;i < n;i++) {
    powerOf2 = powerOf2 * 2;
  }
  // printf("The number of possible partitions is: %d\n",nPartitions);
  // Only half of all possible partitions need be examined
  // The second half is symmetrical to the first half
  cout << "The size of the power set is: " << powerOf2 << endl;
  vector < int > results(powerOf2,0);



  #pragma omp target
  #pragma omp loop
  for(int i = 0;i < (powerOf2);i++) {
    results[i] = evaluateSubSet(i,n,adjMat);
  }
  int n_threads = omp_get_num_threads();
  int solution = 0;


  vector < int > solutionFound (n_threads,0);
  vector < int > maxThisThread(n_threads,0);
  #pragma omp parallel for reduction(max:solution)
  for(int i = 0;i < (powerOf2);i++) {
    if (results[i] > solution) {
      solution = results[i];
      maxThisThread[omp_get_thread_num()] = results[i];
      solutionFound[omp_get_thread_num()] = i;
    }
  }
  cout << "The maximum number of nodes in the independent set is: " << solution << endl;
  for(int i = 0; i < n_threads;i++) {
    if (maxThisThread[i] == solution) {
      printResults(solutionFound[i],n);
    }

  }

}

Overwriting max_ind_set_omp_gpu.cc


In [58]:
!c++ max_ind_set_omp_gpu.cc -o max_ind_set_omp_gpu -fopenmp

# Test matrices

In [11]:
%%writefile k16.txt
16
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k16.txt


In [59]:
!time ./max_ind_set_omp_gpu < k16.txt


The size of the graph is: 16
The number of rows in the adjacency matrix is: 16
The number of columns in the adjacency matrix is: 16
The adjacency matrix is : 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
The size of the power set is: 65536
The maximum number of nodes in the independent set is: 1
An independent set with the maximum number of nodes is: 
0 

real	0m0.038s
user	0m0.032s
sys	0m0.004s


In [44]:
%%writefile test16b.txt
16
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing test16b.txt


In [60]:
!time ./max_ind_set_omp_gpu < test16b.txt

The size of the graph is: 16
The number of rows in the adjacency matrix is: 16
The number of columns in the adjacency matrix is: 16
The adjacency matrix is : 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
The size of the power set is: 65536
The maximum number of nodes in the independent set is: 6
An independent set with the maximum number of nodes is: 
0 1 2 3 4 5 

real	0m0.050s
user	0m0.046s
sys	0m0.002s


In [47]:
%%writefile test16c.txt
16
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0

Writing test16c.txt


In [61]:
!time ./max_ind_set_omp_gpu < test16c.txt

The size of the graph is: 16
The number of rows in the adjacency matrix is: 16
The number of columns in the adjacency matrix is: 16
The adjacency matrix is : 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 
The size of the power set is: 65536
The maximum number of nodes in the independent set is: 6
An independent set with the maximum number of nodes is: 
0 1 2 3 4 5 

real	0m0.062s
user	0m0.055s
sys	0m0.003s


In [64]:
%%writefile k20.txt
20
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Writing k20.txt


In [65]:
!time ./max_ind_set_omp_gpu < k20.txt

The size of the graph is: 20
The number of rows in the adjacency matrix is: 20
The number of columns in the adjacency matrix is: 20
The adjacency matrix is : 
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 
The size of the power